In [19]:
# if the below variable is true,
# it also considers the reply to the msg role: "function" in the training
#conisder_function_summary = False
def process_conversations_to_training_data(conversations, conisder_function_summary = False):
    #our output json data variable
    output_data = []
    process_input = []
    # process each converstion
    prev_role = ""
    for conversation in conversations["conversation"]:
        #we don't train for the input that has no functions
        if conversation['functions'] is None:
            continue

        #get role of the current msg
        cur_role = conversation['chatgptMessage']['role'].lower()

    
        if (cur_role == "assistant" and prev_role != "function") or (cur_role == "assistant" and prev_role == "function" and conisder_function_summary):
            # do an output
            temp_dict = {}
            temp_dict['input'] = process_input.copy()
            temp_dict['target'] = conversation.copy()
            output_data.append(temp_dict)
        # add processed inputs to processed
        process_input.append(conversation)
        prev_role = cur_role

    return output_data

In [20]:
## sample input

import json

with open('production_conversations.json', 'r') as f:
    data = json.load(f)

# sample indx for testing

print(json.dumps(data[35], indent = 4))

{
    "conversationId": "efa5152d-5983-4442-a0df-220888e3b552",
    "conversation": [
        {
            "chatgptMessage": {
                "role": "user",
                "content": "linkedin hashtags for tech"
            },
            "functions": [
                {
                    "name": "getHashtagFollowerCount",
                    "description": "Get the follower count, follower count over time and related trending hashtags of a specified LinkedIn hashtag",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "path_params": {
                                "type": "object",
                                "properties": {
                                    "hashtag": {
                                        "type": "string",
                                        "description": "The specified hashtag."
                                    }
                                },

In [21]:
## sample output
result = process_conversations_to_training_data(data[35], True)
print(json.dumps(result[1], indent = 4))

{
    "input": [
        {
            "chatgptMessage": {
                "role": "user",
                "content": "linkedin hashtags for tech"
            },
            "functions": [
                {
                    "name": "getHashtagFollowerCount",
                    "description": "Get the follower count, follower count over time and related trending hashtags of a specified LinkedIn hashtag",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "path_params": {
                                "type": "object",
                                "properties": {
                                    "hashtag": {
                                        "type": "string",
                                        "description": "The specified hashtag."
                                    }
                                },
                                "required": [
                      

In [22]:
# generate all data
all_data_with_summary = []
for conversation in data:
    all_data_with_summary.extend(process_conversations_to_training_data(conversation, True))

with open('production_data_with_summary.json', 'w') as f:
    json.dump(all_data_with_summary, f)

all_data_without_summary = []
for conversation in data:
    all_data_without_summary.extend(process_conversations_to_training_data(conversation, False))

with open('production_data_without_summary.json', 'w') as f:
    json.dump(all_data_without_summary, f)

In [23]:
print(len(data))
print(len(all_data_with_summary))
print(len(all_data_without_summary))

514
2176
1369
